In [2]:
import os
import sys
sys.path.append('/'.join(os.getcwd().split('/')[:-1]))
flatten = lambda l: [item for sublist in l for item in sublist]

In [3]:
from model.bAbI_utils import bAbI_data_loader, data_loader, pad_to_batch, pad_to_story

test: train_loader

In [4]:
path = '../data/QA_bAbI_tasks/en-10k/qa1_single-supporting-fact_train.txt'
data, word2idx = bAbI_data_loader(path)

In [7]:
batch_size = 32
for batch in data_loader(data, batch_size, shuffle=False):
    break

In [8]:
stories, stories_masks, questions, questions_masks, answers, supports = pad_to_batch(batch, word2idx)

In [6]:
stories.size()

torch.Size([32, 10, 6])

In [7]:
stories_masks.size()

torch.Size([32, 10, 7])

In [8]:
questions.size()

torch.Size([32, 4])

In [9]:
questions_masks.size()

torch.Size([32, 4])

In [10]:
answers.size()

torch.Size([32, 2])

test: test_loader

In [11]:
path_test = '../data/QA_bAbI_tasks/en-10k/qa1_single-supporting-fact_test.txt'
data_test, word2idx = bAbI_data_loader(path_test, word2idx)

In [12]:
batch_size = 32
for batch in data_loader(data_test, batch_size, shuffle=False):
    break

In [24]:
data_test[2]

[[[9, 3, 5, 15, 17],
  [7, 2, 5, 15, 11],
  [18, 4, 14, 5, 15, 11],
  [9, 10, 5, 15, 13],
  [9, 4, 5, 15, 17],
  [19, 2, 5, 15, 20]],
 [12, 16, 19, 21],
 [20],
 8]

In [23]:
stories, stories_masks, questions, questions_masks, answers, supports = pad_to_batch(data_test[2], word2idx)

TypeError: zip argument #4 must support iteration

In [14]:
stories

Variable containing:
(0 ,.,.) = 
   9   3   5  15  17   0
   7   2   5  15  11   0
   0   0   0   0   0   0
   0   0   0   0   0   0

(1 ,.,.) = 
   9   3   5  15  17   0
   7   2   5  15  11   0
  18   4  14   5  15  11
   9  10   5  15  13   0
[torch.LongTensor of size 2x4x6]

In [15]:
stories_masks.size()

torch.Size([32, 10, 7])

In [16]:
questions.size()

torch.Size([32, 4])

In [17]:
questions_masks.size()

torch.Size([32, 4])

In [18]:
answers.size()

torch.Size([32, 2])

test: pad_to_story

In [19]:
stories, stories_masks, questions, questions_masks, answers, supports = pad_to_story(data_test, word2idx)

In [20]:
len(stories), len(stories_masks)

(1000, 1000)

In [21]:
len(questions), len(questions_masks)

(1000, 1000)

In [22]:
len(answers), len(supports)

(1000, 1000)

In [1]:
import torch
from torch.autograd import Variable
from copy import deepcopy
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
flatten = lambda l: [item for sublist in l for item in sublist]

In [72]:
class bAbIDataset(object):
    def __init__(self, path, batch_size, train=True, vocab=None, sos=None, eos=None, return_masks=False):
        self.batch_size = batch_size
        self.train = train
        self.return_masks = return_masks
        if self.train:
            data, vocab = self.bAbI_data_loader(path, vocab=None, sos=sos, eos=eos)
        else:
            data, vocab = self.bAbI_data_loader(path, vocab=vocab, sos=sos, eos=eos)
        
        self.word2idx = vocab
        self.idx2word = {v: k for k, v in self.word2idx.items()}
        self.data = data
                
    def data_loader(self, shuffle=False):
        if shuffle: random.shuffle(self.data)
        sindex = 0
        eindex = self.batch_size
        while eindex < len(self.data):
            batch = self.data[sindex: eindex]
            temp = eindex
            eindex = eindex + self.batch_size
            sindex = temp
            yield batch

        if eindex >= len(self.data):
            batch = self.data[sindex:]
            yield batch
        
    def bAbI_data_loader(self, path, vocab=None, sos=None, eos=None):

        assert ((isinstance(sos, str) and isinstance(eos, str)) == True) or ((sos and eos) is None), print('error')
        try:
            with open(path, 'r', encoding='utf-8') as file:
                data = file.readlines()
            data = [l.strip() for l in data]
        except:
            print('no such file: {}'.format(path))
            return None

        data_temp = []
        story = []

        try:
            for line in data:
                idx, line = line.split(' ', 1)
                if idx == '1':
                    story = []

                if '?' in line:
                    q, a, support = line.split('\t')
                    q = q.lower().strip().replace('?', '').split() + ['?']
                    a = a.lower().strip().split() + [eos] if eos else a.lower().strip().split()
                    support = int(support)
                    story_temp = deepcopy(story)
                    data_temp.append([story_temp, q, a, support])
                else:
                    sentence = line.lower().replace('.', '').split() + [eos] if eos else line.lower().replace('.', '').split()
                    story.append(sentence)
        except:
            print('check data')
            return None

        if vocab:
            data, vocab = self.bAbI_build_vocab(data_temp, vocab)
        else:
            data, vocab = self.bAbI_build_vocab(data_temp, sos=sos, eos=eos)

        return data, vocab

    def bAbI_build_vocab(self, data, vocab=None, **kwargs):
        if vocab is None:
            sos = kwargs['sos']
            eos = kwargs['eos']
            story, q, a, s = list(zip(*data))
            vocab = list(set(flatten(flatten(story)) + flatten(q) + flatten(a)))
            if sos and eos:
                word2idx = {'<pad>': 0, '<unk>': 1, '<s>': 2, '</s>': 3}
            else:
                word2idx = {'<pad>': 0, '<unk>': 1}

            for word in vocab:
                if word2idx.get(word) is None:
                    word2idx[word] = len(word2idx)
        else:
            word2idx = vocab

        for d in data:
            # d[0]: stories
            # d[1]: questions
            # d[2]: answer
            # d[3]: support
            for i, story in enumerate(d[0]):
                d[0][i] = self._transfer2idx(story, word2idx)

            d[1] = self._transfer2idx(d[1], word2idx)
            d[2] = self._transfer2idx(d[2], word2idx)

        return data, word2idx
    
    def _transfer2idx(self, seq, dictionary):
        idxs = list(map(lambda w: dictionary[w] if dictionary.get(w) is not None else dictionary["<unk>"], seq))
        return idxs

    def pad_to_batch(self, batch, w2idx, no_batch=False):
        """
        stories, stories_masks: B, n, T_c
        questions, questions_masks: B, T_q
        answers: B, T_a
        supports: B
        -------------------------------------
        output: stories, stories_masks, questions, questions_masks, answers, supports
        """
        story, q, a, s = list(zip(*batch))
        max_story = max([len(s) for s in story])  # max_stories
        max_len = max([len(s) for s in flatten(story)])  # max_sentence_len
        max_q = max([len(q_) for q_ in q])
        max_a = max([len(a_) for a_ in a])

        stories, stories_masks = [], []
        for i in range(len(batch)):
            story_array, story_mask = self.get_batch_array(self.get_fixed_array(story[i], w2idx), no_batch, max_story, max_len)
            stories.append(story_array)
            stories_masks.append(story_mask)

        questions, questions_masks = self.get_batch_array(self.get_fixed_array(q, w2idx), no_batch, len(batch), max_q)
        answers, _ = self.get_batch_array(self.get_fixed_array(a, w2idx), no_batch, len(batch), max_a)
        
        if self.return_masks:
            return stories, stories_masks, questions, questions_masks, answers, s
        else:
            return stories, questions, answers, s
        
    def get_fixed_array(self, data, w2idx):
        max_col = max([len(d) for d in data])
        for j in range(len(data)):
            if len(data[j]) < max_col:
                data[j].append(w2idx.get('<pad>'))
        return data

    def get_batch_array(self, data, no_batch, *shape):
        assert type(no_batch) == bool, 'no_batch, must be boolean'
        r, c = shape
        if no_batch:
            r = len(data)
        temp = np.zeros((r, c), dtype=np.int)
        it = np.nditer(np.array(data, dtype=np.int), flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            idx = it.multi_index
            tmp_val = np.array(data)[idx]
            temp[idx] = tmp_val
            it.iternext()

        mask = (temp == 0).astype(np.byte)
        return temp.tolist(), mask.tolist()
    
    def trans2tensor(self, x):
        return Variable(torch.LongTensor(x))

    def pad_to_story(test_data, w2idx, no_batch=True):  # this is for inference
        """
        output: stories, stories_masks, questions, questions_masks, answers, supports
        """
        story, q, a, s = list(zip(*test_data))
        max_story = max([len(s) for s in story])  # max_stories
        max_len = max([len(s) for s in flatten(story)])  # max_sentence_len
        max_q = max([len(q_) for q_ in q])
        max_a = max([len(a_) for a_ in a])

        stories, stories_masks = [], []
        for i in range(len(test_data)):
            story_array, story_mask = self.get_batch_array(self.get_fixed_array(story[i], w2idx), no_batch, max_story, max_len)
            stories.append(story_array)
            stories_masks.append(story_mask)

        questions, questions_masks = self.get_batch_array(self.get_fixed_array(q, w2idx), no_batch, len(test_data), max_q)
        answers, _ = self.get_batch_array(self.get_fixed_array(a, w2idx), no_batch, len(test_data), max_a)
        
        if self.return_masks:
            return stories, stories_masks, questions, questions_masks, answers, s
        else:
            return stories, questions, answers, s

In [73]:
bAbI = bAbIDataset(path, train=True)

In [74]:
a = bAbI.pad_to_batch(bAbI.data[:5], bAbI.word2idx)

In [59]:
train_loader = DataLoader(dataset=bAbI, batch_size=32, shuffle=True)

In [55]:
for batch in train_loader:
    break